---
title: Tensor-network error mitigation (TEM) - A Qiskit Function by Algorithmiq
description: Introduction to TEM, a Qiskit Function by Algorithmiq, to compute estimations with software post-processing error mitigation using tensor networks.
---

{/* cspell:ignore POVM, mathbf, Filippov, Lindblad, Leahy, Rossi, García, Pérez, Minev, Kandala, Temme, informationally  */}

# Tensor-network error mitigation (TEM): A Qiskit Function by Algorithmiq

<Admonition type="note" title="Note">
Qiskit Functions are an experimental feature available only to IBM Quantum&reg; Premium Plan, Flex Plan, and On-Prem (via IBM Quantum Platform API) Plan users. They are in preview release status and subject to change.
</Admonition>

{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<Accordion>
<AccordionItem title="Package versions">

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=2.3.0
qiskit-ibm-catalog~=0.11.0
```
</AccordionItem>
</Accordion>

## Overview

Algorithmiq’s Tensor-network Error Mitigation (TEM) method is a hybrid
quantum-classical algorithm designed for performing noise mitigation entirely at
the classical post-processing stage. With TEM, the user can compute the
expectation values of observables mitigating the inevitable noise-induced errors
that occur on quantum hardware with increased accuracy and cost efficiency,
making it a highly attractive option for quantum researchers and industry
practitioners alike.

The method consists of constructing a tensor network representing the inverse of
the global noise channel affecting the state of the quantum processor and then
applying the map to informationally complete measurement outcomes acquired from
the noisy state to obtain unbiased estimators for the observables.

As an advantage, TEM leverages informationally complete measurements to give
access to a vast set of mitigated expectation values of observables and has
optimal sampling overhead on the quantum hardware, as described in Filippov et
al. (2023), [arXiv:2307.11740](https://arxiv.org/abs/2307.11740), and  Filippov
et al. (2024), [arXiv:2403.13542](https://arxiv.org/abs/2403.13542). The
measurement overhead refers to the number of additional measurements required to
perform efficient error mitigation, a critical factor in the feasibility of
quantum computations. Therefore, TEM has the potential to enable quantum
advantage in complex scenarios, such as applications in the fields of quantum
chaos, many-body physics, Hubbard dynamics, and small molecule chemistry
simulations.

The main features and benefits of TEM can be summarized as:

1.  **Optimal measurement overhead**: TEM is optimal with respect to
[theoretical bounds](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.131.210601),
meaning that no method can achieve a smaller measurement overhead. In other
words, TEM requires the minimum number of additional measurements to perform
error mitigation. This in turns means that TEM uses minimal quantum runtime.
2.  **Cost-effectiveness**: Since TEM handles noise mitigation entirely in the
post-processing stage, there is no need to add extra circuits to the quantum
computer, which not only makes the computation cheaper but also diminishes the
risk of introducing additional errors due to the imperfections of quantum
devices.
3.  **Estimation of multiple observables**: Thanks to informationally-complete
measurements, TEM efficiently estimates multiple observables with the same
measurement data from the quantum computer.
4.  **Measurement error mitigation**: The TEM Qiskit Function also includes a
[proprietary measurement error mitigation method](https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.5.033154)
able to significantly reduce readout errors after a short calibration run.
5.  **Accuracy**: TEM significantly improves the accuracy and reliability of
digital quantum simulations, making quantum algorithms more precise and
dependable.

## Description

The TEM function allows you to obtain error-mitigated expectation values for
multiple observables on a quantum circuit with minimal sampling overhead. The
circuit is measured with an informationally complete positive operator-valued
measure (IC-POVM), and the collected measurement outcomes are processed on a
classical computer. This measurement is used to perform the tensor network
methods and build a noise-inversion map. The function applies a map that fully
inverts the whole noisy circuit using tensor networks to represent the noisy
layers.

![TEM schematics](/docs/images/guides/algorithmiq-tem/tem_scheme.svg "Error-mitigated estimation of an observable O via post-processing measurement outcomes of the noisy quantum processor. U and N denote an ideal quantum operation and the associated noise map, which can be generally non-local (and extended to grey boxes). D stands for a tensor of operators that are dual to the effects in the IC measurement. The noise mitigation module M is a tensor network that is efficiently contracted from the middle out. The first iteration of the contraction is represented by the dotted purple line, the second one by the dashed line, and the third one by the solid line.")

Once the circuits are submitted to the function, they are transpiled and
optimized to minimize the number of layers with two-qubit gates (the noisier
gates on quantum devices). The noise affecting the layers is learned through
[Qiskit Runtime](/docs/api/qiskit-ibm-runtime/noise-learner-noise-learner)
using a sparse Pauli-Lindblad noise model as described in E. van den Berg, Z.
Minev, A. Kandala, K. Temme, Nat. Phys. (2023).
[arXiv:2201.09866](https://arxiv.org/abs/2201.09866).

The noise model is an accurate description of the noise on the device able to
capture subtle features, including qubit cross-talk. However, noise on the
devices can fluctuate and drift and the learned noise might not be accurate at
the point at which the estimation is done. This might result in inaccurate
results.

## Get started

Authenticate using your [IBM Quantum Platform API key](http://quantum.cloud.ibm.com/), and select the TEM function as follows. (This snippet assumes you've already [saved your account](/docs/guides/functions#install-qiskit-functions-catalog-client) to your local environment.)

In [ ]:
from qiskit_ibm_catalog import QiskitFunctionsCatalog

tem_function_name = "algorithmiq/tem"
catalog = QiskitFunctionsCatalog(channel="ibm_quantum_platform")

# Load your function
tem = catalog.load(tem_function_name)

## Example


The following snippet shows an example where TEM is used to compute the expectation values of an observable given a simple quantum circuit.

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp

# Create a quantum circuit
qc = QuantumCircuit(3)
qc.u(0.4, 0.9, -0.3, 0)
qc.u(-0.4, 0.2, 1.3, 1)
qc.u(-1.2, -1.2, 0.3, 2)
for _ in range(2):
    qc.barrier()
    qc.cx(0, 1)
    qc.cx(2, 1)
    qc.barrier()
    qc.u(0.4, 0.9, -0.3, 0)
    qc.u(-0.4, 0.2, 1.3, 1)
    qc.u(-1.2, -1.2, 0.3, 2)

# Define the observables
observable = SparsePauliOp("IYX", 1.0)

# Define the execution options
pub = (qc, [observable])
options = {"default_precision": 0.02}

# Define backend to use. TEM will choose the least-busy device reported by IBM if not specified
backend_name = "ibm_marrakesh"

# Run the TEM function (uses around three minutes of QPU time)
job = tem.run(pubs=[pub], backend_name=backend_name, options=options)

Use the Qiskit Serverless APIs to check your Qiskit Function workload's status:

In [3]:
print(job.status())

QUEUED


You can return the results as:

In [7]:
result = job.result()
evs = result[0].data.evs
print(evs[0])

0.10229839785546409


<Admonition type="info">
  The expected value for the noiseless circuit for the given operator should be around `0.18409094298943401`.
</Admonition>

## Inputs

**Parameters**

Name | Type | Description | Required | Default | Example
-- | -- | -- | -- | -- | --
`pubs` | Iterable[EstimatorPubLike] | An iterable of PUB-like (primitive unified bloc) objects, such as tuples `(circuit, observables)` or `(circuit, observables, parameters, precision)`. See [Overview of PUBs](/docs/guides/primitive-input-output#overview-of-pubs) for more information. If a non-ISA circuit is passed, it will be transpiled with optimal settings. If an ISA circuit is passed, it will not be transpiled; in this case, the observable must be defined on the whole QPU. | Yes | N/A | (circuit, observables)
`backend_name` | str | Name of the backend to make the query.| No | If not provided, the least-busy backend will be used. | "ibm_fez"
`options` | dict | Input options. See `Options` section for more details. | No | See `Options` section for more details.| \{"max_bond_dimension": 100\}

<Admonition type="caution">
  TEM currently has the following limitations:

  - Parametrized circuits are not supported. The parameters argument should be set to `None` if precision is specified. This restriction will be removed in future versions.
  - Only circuits without loops are supported. This restriction will be removed in future versions.
  - Non-unitary gates, such as reset, measure, and all forms of control flow are not supported. Support for reset will be added in upcoming releases.
</Admonition>

### Options

A dictionary containing the advanced options for the TEM. The dictionary may contain the keys in the following table. If any of the options are not provided, the default value listed in the table will be used. The default values are good for typical use of TEM.

Name | Type | Description  | Default
-- | -- | -- | --
`tem_max_bond_dimension` | int | The maximum bond dimension for the tensor networks. | 128 |
`tem_compression_cutoff` | float | The cutoff value to be used for the tensor networks. | 1e-10 |
`compute_shadows_bias_from_observable` | bool | Whether the bias for the classical shadows measurement protocol should be tailored to the PUB observable. If False, the classical shadows protocol (equal probability of measuring Z, X, Y) will be used. | False |
`shadows_bias` | np.ndarray | The bias to be used for the randomized classical shadows measurement protocol, a 1D or 2D array of size 3 or shape (num_qubits, 3) respectively. Order is ZXY. | np.array([1 / 3, 1 / 3, 1 / 3]) |
`max_execution_time` | int or None | The maximum execution time on the QPU in seconds. If the runtime exceeds this value, the job will be canceled. If None, the default limit set by Qiskit Runtime will apply. | None |
`num_randomizations` | int | The number of randomizations to be used for noise learning and gate twirling. | 32 |
`max_layers_to_learn` | int | The maximum number of unique layers to learn. | 4 |
`mitigate_readout_error` | bool | Whether to perform readout error mitigation. | True |
`num_readout_calibration_shots` | int | The number of shots to be used for readout error mitigation. | 10000 |
`default_precision` | float | The default precision to be used for any PUBs for which the precision is not specified. | 0.05 |
`default_shots` | int or None | The number of shots to use per PUB. If provided, this overrides the specified precision. Otherwise, the number of shots is estimated to provide the precision requested. | None |
`seed` | int or None | Set the seed for the random number generator for reproducibility. | None |
`shots_per_randomization` | int | The total number of shots to use per random learning circuit. | 128 |
`layer_pair_depths` | list[int] | The circuit depths (measured in number of pairs) to use in learning experiments. | [0, 1, 2, 4, 16, 32] |
`layer_noise_model` | NoiseLearnerResult, Sequence[LayerError], or None | The precomputed noise learner result from a previously executed run. | None |
`private` | bool | Whether the QPU jobs should be private. Setting it to True will prevent subsequent downloads of the experiment data and is recommended for confidential jobs. | False |
`tem_enforce_linear_circuit` | bool | Whether to enforce the circuit to be in linear topology. | False |

#### Precision

The precision of the results can be requested in three ways. The precision can be passed in the PUB, and it will apply to that PUB only. Otherwise, a `default_precision` can be passed in the options as specified above. Finally, for advanced use, the specific number of shots per PUB can be passed in the options with `default_shots`, and it will override any other precision option.

If a `default_precision` or a per-PUB value is passed, the number of shots is estimated in order to achieve the desired precision. This is done based on the chosen QPU parameters and error rates as follows:

- Calculating the error per layered gate (EPLG), which quantifies the noise introduced by layers of two-qubit gates. If backend properties are available, it extracts the relevant fidelity; otherwise, it uses a default value.
- Counting the number of two-qubit gates in the circuit, as these are typically the main contributors to noise.
- Computing a noise prefactor based on the number of two-qubit gates and the EPLG.
- Using the requested precision, it estimates the required number of shots by scaling with the noise prefactor and the inverse of the square of the precision.

This approach ensures that the number of shots is sufficient to achieve the desired precision, taking into account both the circuit structure and the backend's noise characteristics.

## Outputs

A Qiskit [PrimitiveResults](/docs/api/qiskit/qiskit.primitives.PrimitiveResult) containing the TEM-mitigated result. The result for each PUB is returned as a [PubResult](/docs/api/qiskit/qiskit.primitives.PubResult) containing the following fields:


Name |Type | Description
-- | -- | --
data | DataBin | A Qiskit [DataBin](/docs/api/qiskit/qiskit.primitives.DataBin) containing the TEM mitigated observable and its standard error. The DataBin has the following fields: <ul><li>`evs`: The TEM-mitigated observable value.</li><li>`stds`: The standard error of the TEM-mitigated observable.</li></ul>
metadata | dict | A dictionary containing additional results. The dictionary contains the following keys: <ul><li>`"evs_non_mitigated"`: The observable value without error mitigation.</li><li>`"stds_non_mitigated"`: The standard error of the result without error mitigation.</li><li>`"evs_mitigated_no_readout_mitigation"`: The observable value with error mitigation but without readout error mitigation.</li><li>`"stds_mitigated_no_readout_mitigation"`: The standard error of the result with error mitigation but without readout error mitigation.</li><li>`"evs_non_mitigated_with_readout_mitigation"`: The observable value without error mitigation but with readout error mitigation.</li><li>`"stds_non_mitigated_with_readout_mitigation"`: The standard error of the result without error mitigation but with readout error mitigation.</li><li>`"resource_usage"`: A dictionary containing the time resources used by the TEM.  </li></ul>

## Fetching error messages

If your workload status is ERROR, use job.result() to fetch the error message as follows:

In [5]:
print(job.result())

PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(1,), dtype=float64>), stds=np.ndarray(<shape=(1,), dtype=float64>)), metadata={'evs_non_mitigated': array([0.09305761]), 'stds_non_mitigated': array([0.05499456]), 'evs_mitigated_no_readout_mitigation': array([0.09371257]), 'stds_mitigated_no_readout_mitigation': array([0.05546237]), 'evs_non_mitigated_with_readout_mitigation': array([0.10156985]), 'stds_non_mitigated_with_readout_mitigation': array([0.05788876])})], metadata={'resource_usage': {'RUNNING: OPTIMIZING_FOR_HARDWARE': {'CPU_TIME': 1.018068}, 'RUNNING: WAITING_FOR_QPU': {'CPU_TIME': 411.145994}, 'RUNNING: POST_PROCESSING': {'CPU_TIME': 10.018932999999997}, 'RUNNING: EXECUTING_QPU': {'QPU_TIME': 187.0}}})


## Get support

Reach out to [qiskit_ibm@algorithmiq.fi](mailto:qiskit_ibm@algorithmiq.fi)

Be sure to include the following information:
- Qiskit Function Job ID (`qiskit-ibm-catalog`), `job.job_id`
- A detailed description of the issue
- Any relevant error messages or codes
- Steps to reproduce the issue

## Next steps

<Admonition type="tip" title="Recommendations">

- [Request access to Algorithmiq Tensor-network error mitigation](https://quantum.ibm.com/functions?id=4b1b9d76-c18b-4788-b70b-15125111fbe6)

</Admonition>